# Break repeating-key XOR

In [91]:
from base64 import b64decode

In [92]:
def calculateHammingDistance(string_1, string_2):
    string_1_bin = ""
    string_2_bin = ""
    XOR_bin = ""
    for i in range(len(string_1)):
        tmp_bin = bin(ord(string_1[i]))[2:]
        for j in range(8 - len(tmp_bin)):
            tmp_bin = "0" + tmp_bin
        string_1_bin += tmp_bin
    
    for i in range(len(string_2)):
        tmp_bin = bin(ord(string_2[i]))[2:]
        for j in range(8 - len(tmp_bin)):
            tmp_bin = "0" + tmp_bin
        string_2_bin += tmp_bin

    for i in range(len(string_1_bin)):
        if string_1_bin[i] == string_2_bin[i]:
            XOR_bin += "0"
        else:
            XOR_bin += "1"
    
    return XOR_bin.count("1")


In [93]:
def findKeyLength(input_data):
    distacnes_dic = {}
    for key_size in range(2, 41):
        blocks = []
        for i in range(0, len(input_data), key_size):
            blocks.append(input_data[i:i+key_size])    # put the same char (same key char) into the same group
        blocks = blocks[:4] # take 4 KEYSIZE blocks instead of 2 and average the distances
        # Compare the distance between pairs separately
        distance = 0
        counter = 0
        
        for i in range(3):
            for j in range(i+1,4):
                counter += 1
                distance += calculateHammingDistance(blocks[i], blocks[j])
            
        # Calculate the average distance
        distance /= 6

        # Calculate normalized distance and put in the dic
        distacnes_dic[key_size] = distance / key_size

        distacnes_sorted_key = sorted(distacnes_dic,key=distacnes_dic.get)
        distacnes_dic_sorted = {}
        for i in distacnes_sorted_key:
            distacnes_dic_sorted[i] = distacnes_dic[i]

    return distacnes_sorted_key[0]


In [94]:
def scoring(text):
    score = 0
    # a - z: 97 - 112
    # A - Z: 65 - 90
    # 0 - 9: 49 - 57
    # space: 32
    for char in text:
        if ord(char) == 32:
            score += 5
        if 97 <= ord(char)<= 112 or 65 <= ord(char) <= 90 :
            score += 3
        if 49 <= ord(char) <= 57:
            score += 1
    return score


In [95]:
def checkStr(input_hex):
    input_hex_copy =input_hex
    char_list = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789 :;\'\"#$%^&*~`"
    hex_list = []
    bin_str = ""
    score = 0
    max_score = 0
    key = ""
    max_score_char = ""
    max_score_str = ""

    # Group the inputs by two and convert them to binary representation. 
    while len(input_hex) != 0:
        hex_list.append(input_hex[:2])
        input_hex = input_hex[2:]
    
    # Complete the binary bits into eight bits.
    for i in range(len(hex_list)):
        tmp_bin = bin(int(hex_list[i], 16))[2:]
        for j in range(8 - len(tmp_bin)):
            tmp_bin = "0" + tmp_bin
        bin_str += tmp_bin
    
    for char in char_list:
        bin_char = ""
        XOR_bin = ""
        XOR_str = ""
        tmp_bin_char = bin(ord(char))[2:]
        
        # Converts characters to binary
        # Adds the length to the "input" length.
        for i in range(8 - len(tmp_bin_char)):
            tmp_bin_char = "0" + tmp_bin_char
        for i in range(len(bin_str) // 8):
            bin_char += tmp_bin_char
        
        # Perform XOR operations
        for i in range(len(bin_str)):
            if bin_str[i] == bin_char[i]:
                XOR_bin += "0"
            else:
                XOR_bin += "1"
        
        # Converts the result after XOR into characters.
        while len(XOR_bin) != 0:
            XOR_str += chr(int(XOR_bin[:8],2))
            XOR_bin = XOR_bin[8:]

        # Perform scoring
        score = scoring(XOR_str)
        if score > max_score:
            max_score_char = char
            max_score_str = XOR_str
            max_score = score
            key = char
    # print(max_score_str,":",max_score)
    return max_score, max_score_str, input_hex_copy, key

In [96]:
def breakRepeatedKey(input_str, key_len):
    possible_keys = ""
    blocks = []
    for i in range(key_len):
        blocks.append("")
        for j in range(i, len(input_str), key_len):
            tmp_hex = hex(ord(input_str[j]))[2:]
            if len(tmp_hex) == 1:
                tmp_hex = "0" + tmp_hex
            blocks[i] += tmp_hex
        max_score, max_score_str, input_hex, key = checkStr(blocks[i])
        possible_keys += key
    return possible_keys

In [97]:
def XOREncrypt(plaintext, key):
    key_remainder = len(plaintext) % len(key)
    key_padding = ""
    plaintext_bin = ""
    key_bin = ""
    XOR_bin = ""
    ciphertext = ""
    
    # Expand key to text length
    for i in range(len(plaintext) // len(key)):
        key_padding += key
    key_padding += key[:key_remainder]
    
    # Converting text to binary
    for i in range(len(plaintext)):
        tmp_bin = bin(ord(plaintext[i]))[2:]
        for j in range(8 - len(tmp_bin)):
            tmp_bin = "0" + tmp_bin
        plaintext_bin += tmp_bin

    # Converting key to binary
    for i in range(len(key_padding)):
        tmp_bin = bin(ord(key_padding[i]))[2:]
        for j in range(8 - len(tmp_bin)):
            tmp_bin = "0" + tmp_bin
        key_bin += tmp_bin
    
    # XOR binary text and binary key
    for i in range(len(plaintext_bin)):
        if plaintext_bin[i] == key_bin[i]:
            XOR_bin += "0"
        else:
            XOR_bin += "1"

    # Convert the result of XOR to hex
    while len(XOR_bin) != 0:
        tmp_hex = hex(int(XOR_bin[:8],2))[2:]
        if len(tmp_hex) == 1:
            tmp_hex = "0" + tmp_hex
        ciphertext += chr(int(tmp_hex, 16))
        XOR_bin = XOR_bin[8:]
    return ciphertext



In [98]:
def main():
    input_list = []
    input_str = ""
    f = open("Q6.txt","r")
    lines = f.readlines()
    for line in lines: 
        input_list.append(line[0:-1])
    for _str in input_list:
        input_str += b64decode(_str).decode()
    
    key_length = findKeyLength(input_str)
    print("Key Length: ", key_length)

    key = breakRepeatedKey(input_str, key_length)
    print("Key: ", key)
    print()

    plaintext = XOREncrypt(input_str, key)
    print("plaintext: ", plaintext)
    

In [99]:
main()

Key Length:  29
Key:  Terminator X: Bring the noise

plaintext:  I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effec